In [5]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON, RDF, XML
from pandas import json_normalize
import pandas as pd
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
from urllib.parse import quote

In [2]:
### http://UPS-SCLAB-04:7200/repositories/PracticaFinal
endPoint = "https://dbpedia.org/sparql"
sparql = SPARQLWrapper(endPoint)

In [3]:
def get_results(query):
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)
    results = sparql.query().convert()
    return(results)

In [6]:
def receive_broaders_lists(list_broaders,level):

    for i in list_broaders:
      
        resource = quote(i)
        query = """
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX dct: <http://purl.org/dc/terms/>

        select distinct ?level ?r ?c1
            {
                {VALUES (?r ?level) {(dbc:%s + %s)}

                    ?r ^skos:broader{1} ?c1.

                FILTER (!regex(?c1, "lists"))
            }
    
        }    
        """%(resource,str(level))

    resultados = get_results(query)
    df_consulta1 = json_normalize(resultados["results"]["bindings"])
    return df_consulta1 

In [7]:
lista = ["Waste","Pollution_control_technologies","Electronic_waste"]
level = 3
resultados = receive_broaders_lists(lista, level)

In [8]:
print(type(resultados))

<class 'pandas.core.frame.DataFrame'>


In [10]:
resultados

,level.type,level.datatype,level.value,r.type,r.value,c1.type,c1.value
0,typed-literal,http://www.w3.org/2001/XMLSchema#integer,3,uri,http://dbpedia.org/resource/Category:Electroni...,uri,http://dbpedia.org/resource/Category:Computer_...
1,typed-literal,http://www.w3.org/2001/XMLSchema#integer,3,uri,http://dbpedia.org/resource/Category:Electroni...,uri,http://dbpedia.org/resource/Category:Electroni...
2,typed-literal,http://www.w3.org/2001/XMLSchema#integer,3,uri,http://dbpedia.org/resource/Category:Electroni...,uri,http://dbpedia.org/resource/Category:Electroni...
3,typed-literal,http://www.w3.org/2001/XMLSchema#integer,3,uri,http://dbpedia.org/resource/Category:Electroni...,uri,http://dbpedia.org/resource/Category:Electroni...
4,typed-literal,http://www.w3.org/2001/XMLSchema#integer,3,uri,http://dbpedia.org/resource/Category:Electroni...,uri,http://dbpedia.org/resource/Category:Electroni...


In [11]:
# Filter the columns that are of interest.
resultados = resultados[['level.value', 'r.value', 'c1.value']]

resultados.head(3)

,level.value,r.value,c1.value
0,3,http://dbpedia.org/resource/Category:Electroni...,http://dbpedia.org/resource/Category:Computer_...
1,3,http://dbpedia.org/resource/Category:Electroni...,http://dbpedia.org/resource/Category:Electroni...
2,3,http://dbpedia.org/resource/Category:Electroni...,http://dbpedia.org/resource/Category:Electroni...


In [12]:
# Assuming your DataFrame is named df and you want to transform the 'column_name' column
column_list = resultados['c1.value'].tolist()

# Display the resulting list
print(column_list)


['http://dbpedia.org/resource/Category:Computer_recycling', 'http://dbpedia.org/resource/Category:Electronic_waste_in_Africa', 'http://dbpedia.org/resource/Category:Electronic_waste_in_Europe', 'http://dbpedia.org/resource/Category:Electronic_waste_in_Asia', 'http://dbpedia.org/resource/Category:Electronic_waste_by_country']
